In [ ]:
def training(params):

    bk_name_X = params['bk_name_X']
    data_name_X = params['ms_name_X']
    feature_X_list = params['feature_X_list']

    bk_name_y = params['bk_name_y']
    data_name_y = params['ms_name_y']
    feature_y_list = params['feature_y_list']

    train_info = params['train_parameter']
    model_info = params['model_parameter']

    if params['model_name'] is None:
        model_name = None
    else:
        model_name = params['model_name']
    
    model_purpose = params['model_purpose']
    model_method = params['model_method']
    model_tags = params['model_tags']


    ms_name_index = data_name_X.find('_clean')
    collection_name = data_name_X[:ms_name_index]
    data_meta = mongodb_client.get_document_by_json(bk_name_X, collection_name, {'data_name': data_name_X})[0]

    # collection_index = collection_name.find('_')
    # app_name = collection_name[collection_index:]

    if params['model_name'] is None:
        model_name = None
    else:
        model_name = params['model_name']

    if model_purpose == 'Regression' or 'Classification':
        split_mode = 'windows_split'
        data_y_flag = True
    elif model_purpose == 'Forecasting':
        split_mode = 'step_split'
        data_y_flag = False

    # ingestion pipeline
    ingestion_method = 'ms_all'
    ingestion_param_X ={
        "bucket_name": bk_name_X,
        "ms_name" : data_name_X,
        "feature_list":feature_X_list
    }
    ingestion_param_y ={
        "bucket_name": bk_name_y,
        "ms_name" : data_name_y,
        "feature_list":feature_y_list
    }

    data_X, data_y = ML_pipeline.Xy_data_preparation(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

    nan_ratio = 0.005

    data_X = tool.random_nan_df(data_X, nan_ratio)
    data_y = tool.random_nan_df(data_y, nan_ratio)

    scaler_param = 'scale'
    scale_method = 'minmax'
    scaler_path = './scaler/'

    dataX_scaled, X_scalerFilePath, datay_scaled, y_scalerFilePath= ML_pipeline.Xy_data_scaling_train(data_name_X, data_X, data_name_y, data_y, scaler_path, scaler_param, scale_method)

    model_clean = False
    nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}

    dataX_scaled = ML_pipeline.clean_low_quality_column(model_clean, nan_process_info, dataX_scaled)

    split_ratio = 0.8
    day_window_size = tool.get_default_day_window_size(dataX_scaled)
    train_X, val_X, train_y, val_y = ML_pipeline.split_data_by_mode(split_mode, split_ratio, dataX_scaled, datay_scaled, day_window_size)

    max_nan_limit_ratio = 0.5
    if split_mode =='windows_split':
        transform_parameter = {
                'past_step':day_window_size,
                'max_nan_limit_ratio': max_nan_limit_ratio
        }
    else:
        transform_parameter = {
                'future_step': 2,
                'past_step': 24, 
                'max_nan_limit_ratio': max_nan_limit_ratio
        }

    train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transform_parameter, train_X, train_y)
    val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transform_parameter, val_X, val_y)

    # set parameter
    seq_len, input_size = train_X_array.shape[1], train_X_array.shape[2]
    train_parameter = set_train_parameter(train_info)
    model_parameter = set_model_parameter(model_method, model_info, seq_len, input_size)

    # model name & path
    if model_name is None:
        collection_index = collection_name.find('_')
        app_name = collection_name[collection_index:]
        model_name = tool.get_default_model_name(model_name, app_name, model_method, model_clean)
        
    model_file_path = tool.get_default_model_path(model_name, data_name_X, model_method, train_parameter)

    # model training
    if model_purpose == 'Regression' or 'Forecasting':    
        ML_pipeline.CLUST_regresstion_train(train_parameter, model_method, model_parameter, model_file_path, train_X_array, train_y_array, val_X_array, val_y_array)
    elif model_purpose == 'Classification':
        ML_pipeline.CLUST_classification_train(train_X_array, train_y_array, val_X_array, val_y_array, train_parameter, model_method, model_file_path, model_parameter)
    
    # save model meta
    model_info_meta = model_meta_update(data_meta, model_name, split_mode, feature_X_list, feature_y_list, data_y_flag, model_purpose, model_method, model_tags, model_clean, train_parameter, model_parameter, transform_parameter, scaler_param, data_name_X, data_name_y, model_file_path, X_scalerFilePath, y_scalerFilePath)
    ml_meta.save_model_meta_data(mongodb_client, model_info_meta)

    return model_info_meta




def set_train_parameter(train_info):
    train_parameter = {
        'lr': train_info['lr'],
        'weight_decay': train_info['weight_decay'],
        'device': device, 
        'n_epochs': train_info['n_epochs'], 
        'batch_size': train_info['batch_size']
    }
    return train_parameter


def set_model_parameter(model_method, model_info, seq_len, input_size):
    # -------------------- regression & forecasting model method parameter ---------------------------------
    if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
        model_parameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': model_info['hidden_size'],
        'num_layers': model_info['num_layers'],
        'output_dim': model_info['output_dim'], 
        'dropout': model_info['dropout'], 
        'bidirectional': model_info['bidirectional']
    }
    # CNN_1D model parameters
    elif model_method == 'CNN_1D_rg':
        model_parameter = {
        'input_size': input_size,
        'seq_len': seq_len,
        'output_channels': model_info['output_channels'],
        'kernel_size': model_info['kernel_size'],
        'stride': model_info['stride'],
        'padding': model_info['padding'], 
        'dropout': model_info['dropout']
        }
    # LSTM_FCNs model parameters
    elif model_method == 'LSTM_FCNs_rg':
        model_parameter = {
        'input_size': input_size,
        'num_layers': model_info['num_layers'],
        'lstm_dropout': model_info['lstm_dropout'],
        'fc_dropout': model_info['fc_dropout']
        }
    # FC model parameters
    elif model_method == 'FC_rg':
        model_parameter = {
        'input_size': input_size,
        'dropout': model_info['dropout'],
        'bias': model_info['bias']
        }

    # -------------------- classification model method parameter ---------------------------------
    elif model_method == 'LSTM_cf' or model_method == 'GRU_cf':
        model_parameter = {
            'input_size': input_size,
            'seq_len': seq_len,
            'num_classes': model_info['num_classes'],
            'num_layers': model_info['num_layers'],  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
            'hidden_size': model_info['hidden_size'],  # hidden state의 차원, int(default: 64, 범위: 1 이상)
            'dropout': model_info['dropout'],  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
            'bidirectional': model_info['bidirectional']  # 모델의 양방향성 여부, bool(default: True)   
        }
        if model_method == 'LSTM_cf':
            model_parameter['rnn_type'] = 'lstm'
        elif model_method == 'GRU_cf':
            model_parameter['rnn_type'] = 'gru'
            
    # CNN_1D model parameters
    elif model_method == 'CNN_1D_cf':
        model_parameter = {
        'input_size': input_size,
        'seq_len': seq_len,
        'num_classes': model_info['num_classes'],
        'output_channels': model_info['output_channels'], # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        'kernel_size': model_info['kernel_size'], # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
        'stride': model_info['stride'], # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
        'padding': model_info['padding'], # padding 크기, int(default: 0, 범위: 0 이상)
        'drop_out': model_info['drop_out'] # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        }
        
    # LSTM_FCNs model parameters
    elif model_method == 'LSTM_FCNs_cf':
        model_parameter = {
        'input_size': input_size,
        'seq_len': seq_len,
        'num_classes': model_info['num_classes'],
        'num_layers': model_info['num_layers'],  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
        'lstm_drop_out': model_info['lstm_drop_out'], # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
        'fc_drop_out': model_info['fc_drop_out'] # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        }
        
    # FC model parameters
    elif model_method == 'FC_cf':
        model_parameter = {
        'input_size': input_size,
        'num_classes': model_info['num_classes'],
        'drop_out': model_info['drop_out'], # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        'bias': model_info['bias']# bias 사용 여부, bool(default: True)
        }

    return model_parameter



def model_meta_update(data_meta, model_name, split_mode, feature_X_list, feature_y_list, data_y_flag, model_purpose, model_method, model_tags, model_clean, train_parameter, model_parameter, transform_parameter, scaler_param, data_name_X, data_name_y, model_file_path, X_scalerFilePath, y_scalerFilePath):
    model_info_meta ={
        "trainDataInfo":data_meta,
        "modelName":model_name,
        "dataSplitMode": split_mode,
        "featureList":feature_X_list,
        "target": feature_y_list,
        "data_y_flag": data_y_flag,
        "trainDataType":'timeseries',
        "modelPurpose":model_purpose,
        "modelMethod":model_method,
        "modelTags":model_tags,
        "modelCleanLevel":model_clean,
        "trainParameter": train_parameter,
        "modelParameter": model_parameter,
        "transformParameter":transform_parameter,
        "scalerParam":scaler_param,
        "trainDataName":[data_name_X, data_name_y], 

        "files":{
            "modelFile":{
                "fileName":"model.pth",
                "filePath":model_file_path
            },
            "XScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":X_scalerFilePath       
            },
            "yScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":y_scalerFilePath      
            }
        }
    }

    return  model_info_meta
























# def set_train_parameter(n_epochs):
#     train_parameter = {
#         'lr': 1e-4,
#         'weight_decay': 1e-6, 
#         'device': device, 
#         'n_epochs': n_epochs, 
#         'batch_size': 16
#     }

#     return train_parameter


# def set_model_parameter(model_method, seq_len, input_size):
#     # -------------------- regression & forecasting model method parameter ---------------------------------
#     if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
#         model_parameter = {
#         'rnn_type': 'lstm',
#         'input_size': input_size, 
#         'hidden_size': 64,
#         'num_layers': 2,
#         'output_dim': 1, 
#         'dropout': 0.1, 
#         'bidirectional': True
#     }
#     # CNN_1D model parameters
#     elif model_method == 'CNN_1D_rg':
#         model_parameter = {
#         'input_size': input_size,
#         'seq_len': seq_len,
#         'output_channels': 64,
#         'kernel_size': 3,
#         'stride': 1,
#         'padding': 0, 
#         'dropout': 0.1
#         }
#     # LSTM_FCNs model parameters
#     elif model_method == 'LSTM_FCNs_rg':
#         model_parameter = {
#         'input_size': input_size,
#         'num_layers': 2,
#         'lstm_dropout': 0.4,
#         'fc_dropout': 0.1
#         }
#     # FC model parameters
#     elif model_method == 'FC_rg':
#         model_parameter = {
#         'input_size': input_size,
#         'dropout': 0.1,
#         'bias': True
#         }

#     # -------------------- classification model method parameter ---------------------------------
#     elif model_method == 'LSTM_cf' or model_method == 'GRU_cf':
#         model_parameter = {
#             'input_size': input_size,
#             'seq_len': seq_len,
#             'num_classes': 6,
#             'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
#             'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
#             'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
#             'bidirectional': True  # 모델의 양방향성 여부, bool(default: True)   
#         }
#         if model_method == 'LSTM_cf':
#             model_parameter['rnn_type'] = 'lstm'
#         elif model_method == 'GRU_cf':
#             model_parameter['rnn_type'] = 'gru'
            
#     # CNN_1D model parameters
#     elif model_method == 'CNN_1D_cf':
#         model_parameter = {
#         'input_size': input_size,
#         'seq_len': seq_len,
#         'num_classes': 6,
#         'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
#         'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
#         'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
#         'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
#         'drop_out': 0.1 # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
#         }
        
#     # LSTM_FCNs model parameters
#     elif model_method == 'LSTM_FCNs_cf':
#         model_parameter = {
#         'input_size': input_size,
#         'seq_len': seq_len,
#         'num_classes': 6,
#         'num_layers': 1,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
#         'lstm_drop_out': 0.4, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
#         'fc_drop_out': 0.1 # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
#         }
        
#     # FC model parameters
#     elif model_method == 'FC_cf':
#         model_parameter = {
#         'input_size': input_size,
#         'num_classes': 6,
#         'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
#         'bias': True# bias 사용 여부, bool(default: True)
#         }

#     return model_parameter